<a href="https://colab.research.google.com/github/Atria14/data-science/blob/main/04_keras/1_mnist_dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
keras.__version__

'2.9.0'

# Pierwszy przykład sieci neuronowej


W zaprezentowanym przykładzie próbujemy rozwiązać problem klasyfikacji obrazów w skali szarości przedstawiających ręcznie zapisane cyfry (obrazy te mają rozdzielczość 28x28 pikseli). Chcemy podzielić je na 10 kategorii (cyfry od 0 do 9). Będziemy korzystać ze zbioru danych MNIST, który jest uznawany przez środowisko analityków za zbiór klasyczny. Istnieje on tak długo, jak długa jest historia uczenia maszynowego. Zbiór ten zawiera 60 000 obrazów treningowych oraz 10 000 obrazów testowych. Został on utworzony przez Narodowy Instytut Standaryzacji i Technologii (NIST) w latach 80. ubiegłego wieku. Rozwiązanie wspomnianego problemu można porównać do wyświetlenia napisu „Witaj, świecie!” podczas nauki nowego języka programowania. Zbiór ten jest również używany w celu sprawdzania tego, czy algorytm działa poprawnie. Jeżeli zaczniesz zawodowo zajmować się uczeniem maszynowym, to odkryjesz, że zbiór MNIST pojawia się ciągle w różnych pracach naukowych, artykułach publikowanych w internecie itd. Na rysunku 2.1 przedstawiono wybrane elementy tego zbioru.


Zbiór danych MNIST jest dołączony do pakietu Keras w formie czterech tablic Numpy:

In [3]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Tablice train_images i train_labels tworzą treningowy zbiór danych. Będzie on używany podczas trenowania modelu. Do testowania posłuży nam testowy zbiór danych, składający się z tablic test_images i test_labels. Obrazy są zakodowane w formie tablic Numpy, a etykiety mają formę tablicy cyfr (od 0 do 9). Do każdego obrazu przypisana jest tylko jedna etykieta.

Przyjrzyjmy się treningowemu zbiorowi danych:

In [7]:
train_images.shape
train_images[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [8]:
len(train_labels)

60000

In [9]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

A teraz zobaczmy, jak wyglądają dane testowe:

In [10]:
test_images.shape

(10000, 28, 28)

In [11]:
len(test_labels)

10000

In [12]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Będziemy pracować według następującego przepływu roboczego: najpierw będziemy trenować sieć neuronową na danych treningowych: train_images i train_labels. Sieć nauczy się kojarzyć obrazy i etykiety. Następnie nasza sieć wygeneruje przewidywania dotyczące zbioru test_images, a uzyskane wyniki porównamy z etykietami test_labels.


In [13]:
from keras import models
from keras import layers

network = models.Sequential()
#warstwa gęsta z 512 jednostkami wyjściowymi
#Utworzyliśmy warstwę, która przyjmuje na wejściu tylko tensory dwuwymiarowe,
#a pierwszy wymiar tensora musi mieć długość 784 (oś 0 — wymiar próbki — jest 
#nieokreślony, a więc warstwa akceptuje jego dowolną wielkość). Warstwa ta zwróci 
#tensor, którego pierwszy wymiar będzie miał długość równą 512.
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
#W związku z tym warstwa ta może być połączona z kolejną warstwą, 
#Do drugiej warstwy nie przekazaliśmy argumentu definiującego kształt tensora 
#wejściowego — parametr ten zostanie automatycznie określony na podstawie kształtu 
#tensora zwracanego przez wcześniejszą warstwę.


#w związku z tym warstwa taktóra oczekuje na wejściu pojawienia się wektora o 
#rozmiarze 512.
network.add(layers.Dense(10, activation='softmax'))


Głównym blokiem składowym sieci neuronowej jest warstwa (ang. layer). Jest to moduł przetwarzania danych, który można traktować jako filtr danych. Dane wychodzące z filtra mają bardziej przydatną formę od danych do niego wchodzących. Niektóre warstwy dokonują ekstrakcji reprezentacji kierowanych do nich danych — reprezentacje te powinny ułatwiać rozwiązanie problemu, z którym się zmagamy. Większość uczenia głębokiego składa się z łączenia ze sobą prostych warstw w celu zaimplementowania progresywnej destylacji danych. Model uczenia głębokiego jest jak sito przetwarzające dane składające się z coraz drobniejszych siatek — warstw.

Nasza sieć składa się z sekwencji dwóch warstw Dense, które są ze sobą połączone w sposób gęsty (dochodzi tu do gęstego połączenia). Druga warstwa jest dziesięcioelementową warstwą softmax — warstwa ta zwróci tablicę 10 wartości prawdopodobieństwa (suma wszystkich tych wartości jest równa 1). Każdy z tych wyników określa prawdopodobieństwo tego, że na danym obrazie przedstawiono daną cyfrę (obraz może przedstawiać jedną z dziesięciu cyfr).

Wybór właściwej architektury sieci to raczej kwestia doświadczenia niż analizy naukowej. Co prawda są pewne zasady doboru architektury do problemu, z których możesz korzystać, ale tylko praktyka sprawi, że będziesz w stanie zrobić to naprawdę dobrze. 



Trenowanie sieci neuronowej jest związane z rzeczami, którymi są:
* warstwy, które po połączeniu ze sobą tworzą sieć (lub model);
* dane wejściowe i odpowiadające im docelowe etykiety;
* funkcja straty, która definiuje sygnał zwrotny używany w procesie uczenia;
* optymalizator, który określa przebieg trenowania.
Zależności między nimi przedstawiono na rysunku. 

![Alt text](img/1.1.png "a title")



Sieć składająca się z połączonych ze sobą warstw przypisuje przewidywane wartości wyjściowe do danych wejściowych. Następnie funkcja straty porównuje wyniki przewidywań sieci z docelowymi etykietami, wskutek czego obliczana jest wartość straty (miara tego, czy sieć zwraca oczekiwane wartości). Optymalizator korzysta z wartości straty podczas modyfikowania wag sieci.



Tak więc, Na etapie kompilacji musimy określić jeszcze trzy rzeczy w celu przygotowania sieci do trenowania. Są to:

* Funkcja straty (funkcja celu) — funkcja ta definiuje sposób pomiaru wydajności sieci podczas przetwarzania treningowego zbioru danych, a więc pozwala na dostrajanie parametrów sieci we właściwym kierunku.
* Optymalizator — mechanizm dostrajania sieci na podstawie danych zwracanych przez funkcje straty.
* Metryki monitorowane podczas trenowania i testowania — tutaj interesuje nas jedynie dokładność (część obrazów, która została właściwie sklasyfikowana).

Odpowiedni dobór funkcji celu do problemu jest bardzo ważny — sieć będzie robiła wszystko, by zminimalizować straty, a więc jeżeli funkcja celu nie będzie w pełni skorelowana z osiągnięciem sukcesu w wykonywaniu zadania, to sieć będzie wykonywała niechciane operacje. Wyobraź sobie głupią wszechmogącą sztuczną inteligencję trenowaną za pomocą algorytmu SGD przy źle dobranej funkcji celu w postaci „maksymalizuj średni dobrobyt wszystkich żywych ludzi”. Sztuczna inteligencja, aby osiągnąć ten cel w sposób jak najprostszy, może zdecydować się na zabicie wszystkich ludzi poza kilkoma najbogatszymi osobami. Takie rozwiązanie jest możliwe, ponieważ na średni dobrobyt nie wpływa liczba osób pozostałych przy życiu, a prawdopodobnie takiego rozwiązania nie miał na celu autor tej sztucznej inteligencji! Pamiętaj o tym, że wszystkie konstruowane przez Ciebie sztuczne sieci neuronowe będą zachowywały się podobnie — będą starały się za wszelką cenę obniżyć funkcję straty. W związku z tym musisz rozważnie dobierać cele, bo w przeciwnym razie osiągniesz niezamierzone efekty uboczne.

Na szczęście w typowych problemach, takich jak klasyfikacja, regresja i przewidy- wanie sekwencyjne, można korzystać z prostych wskazówek umożliwiających wybranie właściwej funkcji straty. W przypadku podziału na dwie grupy będziemy posługiwać się entropią krzyżową, a w przypadku dzielenia na wiele grup będziemy korzystać z kategoryzacyjnej entropii krzyżowej. Problem regresji będzie rozwiązywany za pomocą średniego błędu kwadratowego, a podczas pracy nad problemem uczenia sekwencyjnego będziemy korzystać z klasyfikacji CTC (ang. Connectionist Temporal Classification). Funkcje celu należy tworzyć samodzielnie w zasadzie tylko podczas pracy nad naprawdę nowym problemem badawczym. 

Optymalizator to sposób modyfikowania sieci na podstawie funkcji straty. Implementuje on określony wariant algorytmu stochastycznego spadku wzdłuż gradientu.

In [15]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


Zanim rozpoczniemy trenowanie, zmienimy kształt danych tak, aby przyjęły kształt oczekiwany przez sieć, i przeskalujemy je do wartości z zakresu [0, 1]. Początkowo nasze obrazy treningowe były zapisywane w postaci macierzy o wymiarach (60000, 28, 28), zawierającej wartości z zakresu [0, 255], i typie uint8. Przekształcamy je w tablicę typu float32 o wymiarach (60000, 28 * 28), zawierającą wartości od 0 do 1.

In [16]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
(print(train_images))

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Musimy dodatkowo zakodować etykiety za pomocą kategorii (proces ten wyjaśnię w rozdziale 3.):

In [17]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [18]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 4s 8ms/step - loss: 0.2588 - accuracy: 0.9254
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.1042 - accuracy: 0.9690
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0687 - accuracy: 0.9796
Epoch 4/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0496 - accuracy: 0.9847
Epoch 5/5
469/469 [==============================] - 4s 7ms/step - loss: 0.0375 - accuracy: 0.9889


Podczas trenowania wyświetlane są dwie wartości: strata sieci jej dokładność (obie wartości dotyczą treningowego zbioru danych).

Podczas trenowania szybko osiągamy dokładność 0,989 (98,9%). Teraz możemy sprawdzić dokładność przetwarzania testowego zbioru danych:


In [19]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0672 - accuracy: 0.9805


In [22]:
print('test_acc:', test_acc)

test_acc: 0.9804999828338623



W przypadku testowego zbioru danych uzyskaliśmy dokładność na poziomie 97,8%, a więc wartość nieco niższą niż dla zbioru treningowego. Różnica między tymi wartościami wynika z nadmiernego dopasowania. Modele uczenia maszynowego mają tendencję do niższej dokładności przetwarzania nowych danych, niż to miało miejsce w przypadku danych treningowych. 




In [23]:
print(test_labels[0])
#print(test_images)
#print([test_images[0]])
network.predict([[test_images[0]]])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


ValueError: ignored